Things to visualize:
- Diagram
    - A map of texas with the 4 cities in zip codes (red for the 4 cities, gray for the rest), white background with labels for the cities, and a red box around Austin, no title or legend
    - Zoom-in map of Austin (78712 zip code) in the center and colored/filled with openness score, 78712 with 100% opacity and the rest with 20% opacity, no title or legend
    - Zoom-in map of Austin (78712 zip code) in the center and black border, the rest of zip codes with 20% opacity, no title or legend, add points of SVI within 78712, no title or legend
    - hand pick a few SVIs and run segmentation 
- Maps
    - maps of big 5 traits for each city separately (4 cities * 5 traits in a matrix)
    - maps of vegetation, sky, building, road, sidewalk for each city separately (4 cities * 5 features in a matrix)
- scatter plot matrix
    - big 5 traits * vegetation, sky, building, road, sidewalk for each city separately ((4 cities + 1 overall) * 5 traits * 5 features in a matrix). (i.e., 5 by 5 matrix for each city)
    

In [16]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import polars as pl
import os
from shapely.geometry import Point
import numpy as np

# [Previous code remains the same until plotting part]

# Create a figure with subplots for each trait
fig, axes = plt.subplots(2, 3, figsize=(20, 15))
fig.patch.set_alpha(0)  # Make figure background transparent
axes = axes.flatten()

# Plot each trait
for idx, trait in enumerate(trait_means):
    ax = axes[idx]
    
    # Make axis background transparent
    ax.patch.set_alpha(0)
    
    # Get the min and max values for consistent color scaling
    vmin = austin_gdf[trait].min()
    vmax = austin_gdf[trait].max()
    
    # Plot all Austin zipcodes with 20% opacity
    base = austin_gdf.plot(ax=ax, 
                          column=trait,
                          cmap='viridis',
                          alpha=0.2,
                          edgecolor='white',
                          linewidth=0.5,
                          vmin=vmin,
                          vmax=vmax)
    
    # Plot 78712 with 100% opacity
    highlight = austin_gdf[ut_mask].plot(ax=ax,
                                       column=trait,
                                       cmap='viridis',
                                       alpha=1.0,
                                       edgecolor='white',
                                       linewidth=0.5,
                                       vmin=vmin,
                                       vmax=vmax)
    
    # Set the plot limits
    ax.set_xlim(center_x - radius, center_x + radius)
    ax.set_ylim(center_y - radius, center_y + radius)
    
    # Customize the plot
    ax.axis('off')
    ax.set_title(trait.replace('_mean', '').capitalize(), fontsize=14, pad=10)

# Remove the empty subplot
axes[-1].remove()

# Adjust layout
plt.tight_layout()

# Save the plot with transparency
os.makedirs('figs/diagam/', exist_ok=True)
plt.savefig('figs/diagam/austin_big5_maps.png', dpi=300, bbox_inches='tight', transparent=True)
plt.close()

# Also create individual maps for each trait
for trait in trait_means:
    fig, ax = plt.subplots(figsize=(10, 10))
    
    # Make both figure and axis background transparent
    fig.patch.set_alpha(0)
    ax.patch.set_alpha(0)
    
    # Get the min and max values for consistent color scaling
    vmin = austin_gdf[trait].min()
    vmax = austin_gdf[trait].max()
    
    # Plot all Austin zipcodes with 20% opacity
    base = austin_gdf.plot(ax=ax, 
                          column=trait,
                          cmap='viridis',
                          alpha=0.2,
                          edgecolor='white',
                          linewidth=0.5,
                          vmin=vmin,
                          vmax=vmax)
    
    # Plot 78712 with 100% opacity
    highlight = austin_gdf[ut_mask].plot(ax=ax,
                                       column=trait,
                                       cmap='viridis',
                                       alpha=1.0,
                                       edgecolor='white',
                                       linewidth=0.5,
                                       vmin=vmin,
                                       vmax=vmax)
    
    # Set the plot limits
    ax.set_xlim(center_x - radius, center_x + radius)
    ax.set_ylim(center_y - radius, center_y + radius)
    
    # Customize the plot
    ax.axis('off')
    
    # Save the plot with transparency
    plt.savefig(f'figs/diagam/austin_{trait.replace("_mean", "")}_map.png', 
                dpi=300, bbox_inches='tight', transparent=True)
    plt.close()

/media/gisense/koichi/miniconda3/envs/personality/lib/python3.11/site-packages/geopandas/geodataframe.py:1459: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  result = super().__getitem__(key)
/media/gisense/koichi/miniconda3/envs/personality/lib/python3.11/site-packages/geopandas/geodataframe.py:1459: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  result = super().__getitem__(key)
/media/gisense/koichi/miniconda3/envs/personality/lib/python3.11/site-packages/geopandas/geodataframe.py:1459: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  result = super().__getitem__(key)
/media/gisense/koichi/miniconda3/envs/personality/lib/python3.11/site-packages/geopandas/geodataframe.py:1459: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  result = super().__getitem__(key)
/media/gisense/koichi/miniconda3/envs/personality/lib/python3.11/site-packages/geopandas/geodataframe.py:145

In [27]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import polars as pl
import os
from shapely.geometry import Point
import numpy as np

# Load zipcode shapefile and filter for Austin
zip_gdf = gpd.read_file('/media/data/personality/cb_2016_us_zcta510_500k/cb_2016_us_zcta510_500k.shp')
austin_mask = zip_gdf['ZCTA5CE10'].str.startswith('787')
austin_gdf = zip_gdf[austin_mask].copy()

# Load personality data
df_zipcode = pl.read_parquet('data/processed/model/final_dataset_zipcode.parquet')

# Get the Big 5 trait columns
big_five_traits = ['extraversion', 'agreeableness', 'conscientiousness', 'neuroticism', 'openness']
trait_means = [f"{trait}_mean" for trait in big_five_traits]

# Define color palettes for each trait
trait_palettes = {
    'openness_mean': 'Oranges',
    'conscientiousness_mean': 'Greens', 
    'extraversion_mean': 'Reds',
    'agreeableness_mean': 'Blues',
    'neuroticism_mean': 'Purples'
}

# Convert to pandas for easier merging
df_zipcode_pd = df_zipcode.select(['now_zip'] + trait_means).to_pandas()

# Merge with geodataframe
austin_gdf = austin_gdf.merge(df_zipcode_pd, 
                             left_on='ZCTA5CE10', 
                             right_on='now_zip', 
                             how='left')

# Convert to Web Mercator projection for basemap
austin_gdf = austin_gdf.to_crs(epsg=3857)

# Create two layers: one for all Austin zipcodes (20% opacity)
# and one for 78712 (100% opacity)
ut_mask = austin_gdf['ZCTA5CE10'] == '78712'

# Get the bounds of UT area
ut_bounds = austin_gdf[ut_mask].geometry.total_bounds
padding = 2000  # Padding in meters for Web Mercator

# Calculate circle parameters
center_x = (ut_bounds[0] + ut_bounds[2]) / 2
center_y = (ut_bounds[1] + ut_bounds[3]) / 2
radius = min(ut_bounds[2] - ut_bounds[0], ut_bounds[3] - ut_bounds[1]) / 2 + padding/2

# Create circular buffer
center_point = Point(center_x, center_y)
circle_buffer = center_point.buffer(radius)

# Crop GeoDataFrame to circle
austin_gdf['geometry'] = austin_gdf.geometry.intersection(circle_buffer)
austin_gdf = austin_gdf[~austin_gdf.geometry.is_empty]


# Create a figure with subplots for each trait
fig, axes = plt.subplots(2, 3, figsize=(20, 15))
fig.patch.set_alpha(0)  # Make figure background transparent
axes = axes.flatten()

# Plot each trait
for idx, trait in enumerate(trait_means):
    ax = axes[idx]
    
    # Make axis background transparent
    ax.patch.set_alpha(0)
    
    # Get the min and max values for consistent color scaling
    vmin = austin_gdf[trait].min()
    vmax = austin_gdf[trait].max()
    
    # Plot all Austin zipcodes with 20% opacity
    base = austin_gdf.plot(ax=ax, 
                          column=trait,
                          cmap=trait_palettes[trait],
                          alpha=0.2,
                          edgecolor='white',
                          linewidth=0.5,
                          vmin=vmin,
                          vmax=vmax)
    
    # Plot 78712 with 100% opacity
    highlight = austin_gdf[ut_mask].plot(ax=ax,
                                       column=trait,
                                       cmap=trait_palettes[trait],
                                       alpha=1.0,
                                       edgecolor='white',
                                       linewidth=0.5,
                                       vmin=vmin,
                                       vmax=vmax)
    
    # Set the plot limits
    ax.set_xlim(center_x - radius, center_x + radius)
    ax.set_ylim(center_y - radius, center_y + radius)
    
    # Customize the plot
    ax.axis('off')
    ax.set_title(trait.replace('_mean', '').capitalize(), fontsize=14, pad=10)

# Remove the empty subplot
axes[-1].remove()

# Adjust layout
plt.tight_layout()

# Save the plot with transparency
os.makedirs('figs/diagam/', exist_ok=True)
plt.savefig('figs/diagam/austin_big5_maps.png', dpi=300, bbox_inches='tight', transparent=True)
plt.close()

# Also create individual maps for each trait
for trait in trait_means:
    fig, ax = plt.subplots(figsize=(10, 10))
    
    # Make both figure and axis background transparent
    fig.patch.set_alpha(0)
    ax.patch.set_alpha(0)
    
    # Get the min and max values for consistent color scaling
    vmin = austin_gdf[trait].min()
    vmax = austin_gdf[trait].max()
    
    # Plot all Austin zipcodes with 20% opacity
    base = austin_gdf.plot(ax=ax, 
                          column=trait,
                          cmap=trait_palettes[trait],
                          alpha=0.7,
                          edgecolor='white',
                          linewidth=0.5,
                          vmin=vmin,
                          vmax=vmax)
    
    # Plot 78712 with 100% opacity
    highlight = austin_gdf[ut_mask].plot(ax=ax,
                                       column=trait,
                                       cmap=trait_palettes[trait],
                                       alpha=1.0,
                                       edgecolor='white',
                                       linewidth=0.5,
                                       vmin=vmin,
                                       vmax=vmax)
    
    # Set the plot limits
    ax.set_xlim(center_x - radius, center_x + radius)
    ax.set_ylim(center_y - radius, center_y + radius)
    
    # Customize the plot
    ax.axis('off')
    
    # Save the plot with transparency
    plt.savefig(f'figs/diagam/austin_{trait.replace("_mean", "")}_map.png', 
                dpi=300, bbox_inches='tight', transparent=True)
    plt.close()

/media/gisense/koichi/miniconda3/envs/personality/lib/python3.11/site-packages/geopandas/geodataframe.py:1459: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  result = super().__getitem__(key)
/media/gisense/koichi/miniconda3/envs/personality/lib/python3.11/site-packages/geopandas/geodataframe.py:1459: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  result = super().__getitem__(key)
/media/gisense/koichi/miniconda3/envs/personality/lib/python3.11/site-packages/geopandas/geodataframe.py:1459: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  result = super().__getitem__(key)
/media/gisense/koichi/miniconda3/envs/personality/lib/python3.11/site-packages/geopandas/geodataframe.py:1459: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  result = super().__getitem__(key)
/media/gisense/koichi/miniconda3/envs/personality/lib/python3.11/site-packages/geopandas/geodataframe.py:145

In [26]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import polars as pl
import os
from shapely.geometry import Point
import numpy as np

# Load zipcode shapefile and filter for Austin
zip_gdf = gpd.read_file('/media/data/personality/cb_2016_us_zcta510_500k/cb_2016_us_zcta510_500k.shp')
austin_mask = zip_gdf['ZCTA5CE10'].str.startswith('787')
austin_gdf = zip_gdf[austin_mask].copy()

# Load GSV points
gsv_points = pd.read_csv('data/raw/austin_texas/gsv_pids.csv')

# Convert GSV points to GeoDataFrame
gsv_gdf = gpd.GeoDataFrame(
    gsv_points, 
    geometry=gpd.points_from_xy(gsv_points.lon, gsv_points.lat),
    crs="EPSG:4326"
)

# Convert both GeoDataFrames to Web Mercator projection
austin_gdf = austin_gdf.to_crs(epsg=3857)
gsv_gdf = gsv_gdf.to_crs(epsg=3857)

# Create two layers: one for all Austin zipcodes (20% opacity)
# and one for 78712 (black border)
ut_mask = austin_gdf['ZCTA5CE10'] == '78712'

# Get the bounds of UT area
ut_bounds = austin_gdf[ut_mask].geometry.total_bounds
padding = 2000  # Padding in meters for Web Mercator

# Calculate circle parameters
center_x = (ut_bounds[0] + ut_bounds[2]) / 2
center_y = (ut_bounds[1] + ut_bounds[3]) / 2
radius = min(ut_bounds[2] - ut_bounds[0], ut_bounds[3] - ut_bounds[1]) / 2 + padding/2

# Create circular buffer
center_point = Point(center_x, center_y)
circle_buffer = center_point.buffer(radius)

# Crop GeoDataFrame to circle
austin_gdf['geometry'] = austin_gdf.geometry.intersection(circle_buffer)
austin_gdf = austin_gdf[~austin_gdf.geometry.is_empty]

# Create the plot
fig, ax = plt.subplots(figsize=(10, 10))

# Make both figure and axis background transparent
fig.patch.set_alpha(0)
ax.patch.set_alpha(0)

# Plot all Austin zipcodes with 20% opacity and gray border
austin_gdf.plot(ax=ax, 
                color='lightgray',
                alpha=0.7,
                edgecolor='gray',
                linewidth=1.5)

# Plot 78712 with black border
austin_gdf[ut_mask].plot(ax=ax,
                        color='none',
                        edgecolor='black',
                        linewidth=2)

# Get points within 78712
ut_polygon = austin_gdf[ut_mask].geometry.iloc[0]
mask_within = gsv_gdf.within(ut_polygon)
points_within = gsv_gdf[mask_within]

# Plot points within 78712
points_within.plot(ax=ax,
                  color='red',
                  markersize=1,
                  alpha=0.9)

# Set the plot limits
ax.set_xlim(center_x - radius, center_x + radius)
ax.set_ylim(center_y - radius, center_y + radius)

# Customize the plot
ax.axis('off')

# Save the plot with transparency
os.makedirs('figs/diagam/', exist_ok=True)
plt.savefig('figs/diagam/austin_svi_map.png', dpi=300, bbox_inches='tight', transparent=True)
plt.close()

/media/gisense/koichi/miniconda3/envs/personality/lib/python3.11/site-packages/geopandas/geodataframe.py:1459: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  result = super().__getitem__(key)
/media/gisense/koichi/miniconda3/envs/personality/lib/python3.11/site-packages/geopandas/geodataframe.py:1459: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  result = super().__getitem__(key)


In [9]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import polars as pl
import os
import contextily as ctx
import numpy as np
import shutil
from pathlib import Path
import glob

# Create output directory
os.makedirs('figs/diagam/images/', exist_ok=True)

# Load zipcode shapefile and filter for Austin
zip_gdf = gpd.read_file('/media/data/personality/cb_2016_us_zcta510_500k/cb_2016_us_zcta510_500k.shp')
austin_mask = zip_gdf['ZCTA5CE10'].str.startswith('787')
austin_gdf = zip_gdf[austin_mask].copy()

# Load GSV points
gsv_points = pd.read_csv('data/raw/austin_texas/gsv_pids.csv')

# Convert GSV points to GeoDataFrame
gsv_gdf = gpd.GeoDataFrame(
    gsv_points, 
    geometry=gpd.points_from_xy(gsv_points.lon, gsv_points.lat),
    crs="EPSG:4326"
)

# Convert both GeoDataFrames to Web Mercator projection
austin_gdf = austin_gdf.to_crs(epsg=3857)
gsv_gdf = gsv_gdf.to_crs(epsg=3857)

# Create the plot
fig, ax = plt.subplots(figsize=(10, 10))

# Create two layers: one for all Austin zipcodes (20% opacity)
# and one for 78712 (black border)
ut_mask = austin_gdf['ZCTA5CE10'] == '78712'

# Plot all Austin zipcodes with 20% opacity and gray border
austin_gdf.plot(ax=ax, 
                color='lightgray',
                alpha=0.2,
                edgecolor='gray',
                linewidth=1.5)

# Plot 78712 with black border
austin_gdf[ut_mask].plot(ax=ax,
                        color='none',
                        edgecolor='black',
                        linewidth=2)

# Get points within 78712
ut_polygon = austin_gdf[ut_mask].geometry.iloc[0]
mask_within = gsv_gdf.within(ut_polygon)
points_within = gsv_gdf[mask_within]

# Randomly select 5 points
np.random.seed(42)  # for reproducibility
selected_indices = np.random.choice(len(points_within), 5, replace=False)
selected_points = points_within.iloc[selected_indices]

# Plot all points within 78712
points_within.plot(ax=ax,
                  color='red',
                  markersize=1,
                  alpha=0.5)

# Plot selected points with larger markers
selected_points.plot(ax=ax,
                    color='blue',
                    markersize=50,
                    alpha=0.7)

# Add labels for selected points
for idx, point in selected_points.iterrows():
    plt.annotate(point['panoid'],
                xy=(point.geometry.x, point.geometry.y),
                xytext=(10, 10),
                textcoords='offset points',
                fontsize=8,
                bbox=dict(facecolor='white', edgecolor='none', alpha=0.7))

# Zoom to 78712 with some padding
ut_bounds = austin_gdf[ut_mask].geometry.total_bounds
padding = 2000  # Padding in meters for Web Mercator
ax.set_xlim(ut_bounds[0] - padding, ut_bounds[2] + padding)
ax.set_ylim(ut_bounds[1] - padding, ut_bounds[3] + padding)

# Add minimalist basemap
ctx.add_basemap(ax, 
                source=ctx.providers.CartoDB.Positron,
                alpha=0.5)

# Customize the plot
ax.axis('off')
ax.set_facecolor('white')
fig.patch.set_facecolor('white')

# Save the plot
plt.savefig('figs/diagam/selected_points_map.png', dpi=300, bbox_inches='tight')
plt.close()

# Copy selected images
print("Selected panoids:")
base_dir = Path('/media/data/streetview/austin_texas/gsv_panorama')

for idx, point in selected_points.iterrows():
    panoid = point['panoid']
    print(f"- {panoid}")
    
    # Search for image recursively
    found_images = list(base_dir.rglob(f"{panoid}.jpg"))
    
    if found_images:
        source_path = found_images[0]
        target_path = f"figs/diagam/images/{panoid}.jpg"
        shutil.copy2(source_path, target_path)
    else:
        print(f"Warning: Image not found for panoid {panoid}")

Selected panoids:
- luKs6RymNarh0GAFLVY1pw
- Wk2qEWkjfMNiJOAznYUWsw
- qJO4Lm4b07EHeZydYs5Hag
- ekjvORMB0SyPdeqsfqMk8w
- ZlnLEMh8o-hLZTqCuEGxhw


In [10]:
from zensvi.cv import Segmenter

segmenter = Segmenter(dataset = "mapillary")
segmenter.segment("figs/diagam/images", dir_image_output="figs/diagam/images_segmented")



Using GPU


preprocessor_config.json:   0%|          | 0.00/536 [00:00<?, ?B/s]

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.48, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.
/media/gisense/koichi/miniconda3/envs/personality/lib/python3.11/site-packages/transformers/utils/deprecation.py:165: UserWarning: The following named arguments are not valid for `Mask2FormerImageProcessor.__init__` and were ignored: '_max_size'
  return func(*args, **kwargs)


config.json:   0%|          | 0.00/79.5k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/866M [00:00<?, ?B/s]

Processing outer batches of size 5:   0%|          | 0/1 [00:00<?, ?it/s]

Processing outer batch #1:   0%|          | 0/5 [00:00<?, ?it/s]

In [30]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import polars as pl
import os
from pathlib import Path
import numpy as np

# Create output directory
os.makedirs('figs/city_maps/', exist_ok=True)

# Load zipcode shapefile
zip_gdf = gpd.read_file('/media/data/personality/cb_2016_us_zcta510_500k/cb_2016_us_zcta510_500k.shp')

# Load personality data
df_zipcode = pl.read_parquet('data/processed/model/final_dataset_zipcode.parquet')

# Get the Big 5 trait columns and cities
big_five_traits = ['extraversion', 'agreeableness', 'conscientiousness', 'neuroticism', 'openness']
trait_means = [f"{trait}_mean" for trait in big_five_traits]

# Clean up city names by removing "_texas" and converting to sentence case
cities = [city.replace('_texas', '').title() for city in df_zipcode.select('city').unique().to_pandas()['city'].tolist()]

# Define color palettes for each trait
trait_palettes = {
    'openness_mean': 'Oranges',
    'conscientiousness_mean': 'Greens', 
    'extraversion_mean': 'Reds',
    'agreeableness_mean': 'Blues',
    'neuroticism_mean': 'Purples'
}

# Convert to pandas for easier merging
df_zipcode_pd = df_zipcode.select(['now_zip', 'city'] + trait_means).to_pandas()

# Create a large figure for all cities and traits
fig = plt.figure(figsize=(25, 20))
fig.patch.set_alpha(0)  # Make figure background transparent

# Calculate min and max for each trait across the 4 cities only
city_mins = {}
city_maxs = {}

# First pass to calculate min/max values for the 4 cities
for trait in trait_means:
    city_data = df_zipcode_pd[df_zipcode_pd['city'].str.contains('_texas')]
    city_mins[trait] = city_data[trait].min()
    city_maxs[trait] = city_data[trait].max()

# Create grid of subplots
gs = fig.add_gridspec(5, 5, height_ratios=[4, 4, 4, 4, 0.3], hspace=0.3, wspace=0.1)

# Plot each city and trait combination
for city_idx, city in enumerate(cities):
    # Filter data for current city
    city_data = df_zipcode_pd[df_zipcode_pd['city'] == f"{city.lower()}_texas"]
    
    # Get zip codes for the current city
    city_zips = city_data['now_zip'].unique()
    city_mask = zip_gdf['ZCTA5CE10'].isin(city_zips)
    city_gdf = zip_gdf[city_mask].copy()
    
    # Merge with personality data
    city_gdf = city_gdf.merge(city_data, 
                             left_on='ZCTA5CE10', 
                             right_on='now_zip', 
                             how='left')
    
    # Convert to Web Mercator projection
    city_gdf = city_gdf.to_crs(epsg=3857)
    
    # Get the bounds of the city
    city_bounds = city_gdf.geometry.total_bounds
    padding = 2000  # Padding in meters for Web Mercator
    
    # Calculate center and radius for consistent plotting
    center_x = (city_bounds[0] + city_bounds[2]) / 2
    center_y = (city_bounds[1] + city_bounds[3]) / 2
    radius = max(city_bounds[2] - city_bounds[0], city_bounds[3] - city_bounds[1]) / 2 + padding
    
    # Plot each trait
    for trait_idx, trait in enumerate(trait_means):
        ax = fig.add_subplot(gs[city_idx, trait_idx])
        
        # Make axis background transparent
        ax.patch.set_alpha(0)
        
        # Plot zipcodes
        city_gdf.plot(ax=ax, 
                     column=trait,
                     cmap=trait_palettes[trait],
                     alpha=1.0,
                     edgecolor='white',
                     linewidth=0.5,
                     vmin=city_mins[trait],
                     vmax=city_maxs[trait])
        
        # Set the plot limits
        ax.set_xlim(center_x - radius, center_x + radius)
        ax.set_ylim(center_y - radius, center_y + radius)
        
        # Customize the plot
        ax.axis('off')
        
        # Add titles only at the top and left with larger font sizes
        if city_idx == 0:  # Top row gets trait names
            ax.set_title(trait.replace('_mean', '').capitalize(), fontsize=18, pad=10)
        if trait_idx == 0:  # Left column gets city names
            ax.text(-0.2, 0.5, city, rotation=90, 
                   transform=ax.transAxes, fontsize=18, 
                   verticalalignment='center')

# Add colorbars at the bottom
for trait_idx, trait in enumerate(trait_means):
    ax_colorbar = fig.add_subplot(gs[-1, trait_idx])
    norm = plt.Normalize(city_mins[trait], city_maxs[trait])
    sm = plt.cm.ScalarMappable(cmap=trait_palettes[trait], norm=norm)
    sm.set_array([])
    plt.colorbar(sm, cax=ax_colorbar, orientation='horizontal')
    ax_colorbar.set_title(f"{trait.replace('_mean', '').capitalize()} Score", 
                         fontsize=10, pad=10)

# Adjust layout
plt.savefig('figs/city_maps/all_cities_traits_matrix.png', 
            dpi=300, bbox_inches='tight', transparent=True)
plt.close()

In [ ]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import polars as pl
import os
from pathlib import Path
import numpy as np
from matplotlib.colors import LinearSegmentedColormap

# Create output directory
os.makedirs('figs/city_maps/', exist_ok=True)

# Load zipcode shapefile
zip_gdf = gpd.read_file('/media/data/personality/cb_2016_us_zcta510_500k/cb_2016_us_zcta510_500k.shp')

# Load personality data
df_zipcode = pl.read_parquet('data/processed/model/final_dataset_zipcode.parquet')

# convert column names to lowercase
df_zipcode = df_zipcode.select(
    [pl.col(col).rename(col.lower()) for col in df_zipcode.columns]
)

# Get the feature columns and cities
features = ['vegetation', 'sky', 'building', 'road', 'sidewalk']
feature_means = [f"{feature}_ratios_mean" for feature in features]

# Clean up city names by removing "_texas" and converting to sentence case
cities = [city.replace('_texas', '').title() for city in df_zipcode.select('city').unique().to_pandas()['city'].tolist()]

# Define custom color palettes for each feature
feature_colors = {
    'vegetation_ratios_mean': {'color': (107/255, 142/255, 35/255), 'label': 'Vegetation'},
    'sky_ratios_mean': {'color': (70/255, 130/255, 180/255), 'label': 'Sky'},
    'building_ratios_mean': {'color': (70/255, 70/255, 70/255), 'label': 'Building'},
    'road_ratios_mean': {'color': (128/255, 64/255, 128/255), 'label': 'Road'},
    'sidewalk_ratios_mean': {'color': (244/255, 35/255, 232/255), 'label': 'Sidewalk'}
}

# Create custom colormaps
feature_cmaps = {}
for feature, info in feature_colors.items():
    feature_cmaps[feature] = LinearSegmentedColormap.from_list(
        feature,
        ['white', info['color']]
    )

# Convert to pandas for easier merging
df_zipcode_pd = df_zipcode.select(['now_zip', 'city'] + feature_means).to_pandas()

# Create a large figure for all cities and features
fig = plt.figure(figsize=(25, 20))
fig.patch.set_alpha(0)  # Make figure background transparent

# Calculate min and max for each feature across the 4 cities only
city_mins = {}
city_maxs = {}

# First pass to calculate min/max values for the 4 cities
for feature in feature_means:
    city_data = df_zipcode_pd[df_zipcode_pd['city'].str.contains('_texas')]
    city_mins[feature] = city_data[feature].min()
    city_maxs[feature] = city_data[feature].max()

# Create grid of subplots
gs = fig.add_gridspec(5, 5, height_ratios=[4, 4, 4, 4, 0.3], hspace=0.3, wspace=0.1)

# Plot each city and feature combination
for city_idx, city in enumerate(cities):
    # Filter data for current city
    city_data = df_zipcode_pd[df_zipcode_pd['city'] == f"{city.lower()}_texas"]
    
    # Get zip codes for the current city
    city_zips = city_data['now_zip'].unique()
    city_mask = zip_gdf['ZCTA5CE10'].isin(city_zips)
    city_gdf = zip_gdf[city_mask].copy()
    
    # Merge with feature data
    city_gdf = city_gdf.merge(city_data, 
                             left_on='ZCTA5CE10', 
                             right_on='now_zip', 
                             how='left')
    
    # Convert to Web Mercator projection
    city_gdf = city_gdf.to_crs(epsg=3857)
    
    # Get the bounds of the city
    city_bounds = city_gdf.geometry.total_bounds
    padding = 2000  # Padding in meters for Web Mercator
    
    # Calculate center and radius for consistent plotting
    center_x = (city_bounds[0] + city_bounds[2]) / 2
    center_y = (city_bounds[1] + city_bounds[3]) / 2
    radius = max(city_bounds[2] - city_bounds[0], city_bounds[3] - city_bounds[1]) / 2 + padding
    
    # Plot each feature
    for feature_idx, feature in enumerate(feature_means):
        ax = fig.add_subplot(gs[city_idx, feature_idx])
        
        # Make axis background transparent
        ax.patch.set_alpha(0)
        
        # Plot zipcodes
        city_gdf.plot(ax=ax, 
                     column=feature,
                     cmap=feature_cmaps[feature],
                     alpha=1.0,
                     edgecolor='white',
                     linewidth=0.5,
                     vmin=city_mins[feature],
                     vmax=city_maxs[feature])
        
        # Set the plot limits
        ax.set_xlim(center_x - radius, center_x + radius)
        ax.set_ylim(center_y - radius, center_y + radius)
        
        # Customize the plot
        ax.axis('off')
        
        # Add titles only at the top and left with larger font sizes
        if city_idx == 0:  # Top row gets feature names
            ax.set_title(feature_colors[feature]['label'], fontsize=18, pad=10)
        if feature_idx == 0:  # Left column gets city names
            ax.text(-0.2, 0.5, city, rotation=90, 
                   transform=ax.transAxes, fontsize=18, 
                   verticalalignment='center')

# Add colorbars at the bottom
for feature_idx, feature in enumerate(feature_means):
    ax_colorbar = fig.add_subplot(gs[-1, feature_idx])
    norm = plt.Normalize(city_mins[feature], city_maxs[feature])
    sm = plt.cm.ScalarMappable(cmap=feature_cmaps[feature], norm=norm)
    sm.set_array([])
    plt.colorbar(sm, cax=ax_colorbar, orientation='horizontal')
    ax_colorbar.set_title(f"{feature_colors[feature]['label']} Score", 
                         fontsize=10, pad=10)

# Adjust layout
plt.savefig('figs/city_maps/all_cities_features_matrix.png', 
            dpi=300, bbox_inches='tight', transparent=True)
plt.close()